In [5]:
import os
import pickle
import numpy as np
import pandas as pd
from google.cloud import storage
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split

# download and pre-process dataset to remove 'nParticles'
BUCKET_NAME = "cs229-smoua-stacking"
DATASET_PATH = "combined_dataset_labeled.csv"

def download_from_gcs(bucket_name, source_blob_name, destination_file_name):
    """Downloads a file from GCS."""
    client = storage.Client()
    bucket = client.bucket(bucket_name)
    blob = bucket.blob(source_blob_name)
    blob.download_to_filename(destination_file_name)
    print(f"✅ {source_blob_name} downloaded to {destination_file_name}")

download_from_gcs(BUCKET_NAME, DATASET_PATH, "combined_dataset_labeled.csv")

df = pd.read_csv("combined_dataset_labeled.csv")
df.columns = df.columns.str.strip()  # Remove accidental spaces

if "nParticles" in df.columns:
    df = df.drop(columns=["nParticles"])

LABEL_COLUMN = "label"
EVENT_TYPE_COLUMN = "event_type"

X = df.drop(columns=[LABEL_COLUMN, EVENT_TYPE_COLUMN])
y = df[LABEL_COLUMN]
event_types = df[EVENT_TYPE_COLUMN]

event_weights = {
    "HH": 0.0015552 * 1.155 * 5, "ZZ": 0.17088 * 1.155 * 5, "ZH": 0.00207445 * 1.155 * 5,
    "WW": 0.5149 * 5, "tt": 0.503 * 5, "qqX": 0.04347826 * 5, "qqqqX": 0.04 * 5, "qqHX": 0.001 * 5,
    "qq": 0.0349 * 5, "pebb": 0.7536 * 5, "pebbqq": 0.1522 * 5, "peqqH": 0.1237 * 5, "pett": 0.0570 * 5}

X_train, X_test, y_train, y_test, event_train, event_test = train_test_split(
    X, y, event_types, test_size=0.2, random_state=50, stratify=y
)

✅ combined_dataset_labeled.csv downloaded to combined_dataset_labeled.csv


In [3]:
# find ideal hyperparameters for base models

In [ ]:
# xgboost model

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
import xgboost as xgb

scale_pos_weight = np.sum(y_train == 0) / np.sum(y_train == 1)  # Background / Signal -- adjusts for class imbalance

# search grid for optimal parameters
param_grid_xgb = {
    "n_estimators": [50, 100, 200, 300],
    "max_depth": [3, 5, 7, 10],
    "learning_rate": [0.01, 0.05, 0.1],
    "colsample_bytree": [0.7, 0.8, 1.0],
    "subsample": [0.7, 0.8, 1.0],
    "gamma": [0, 0.1, 0.2],
    "reg_lambda": [1, 5, 10], 
    "scale_pos_weight": [scale_pos_weight] 
}

xgb_model = xgb.XGBClassifier()

random_search_xgb = RandomizedSearchCV(
    estimator=xgb_model,
    param_distributions=param_grid_xgb,
    n_iter=20,  
    scoring="recall", # optimize recall for signal detection
    cv=3, 
    verbose=1,
    n_jobs=-1
)

random_search_xgb.fit(X_train, y_train)
print("Best XGBoost Parameters:", random_search_xgb.best_params_)


In [ ]:
# save model
with open("random_search_xgb.pkl", "wb") as model_file:
    pickle.dump(random_search_xgb.best_estimator_, model_file)

print("Fitted and saved XGBoost model'")

In [8]:
from sklearn.metrics import accuracy_score, recall_score, f1_score

def optimize_threshold(model, X_test, y_test, event_test, event_weights, num_thresholds=50):
    y_scores = model.predict_proba(X_test)[:, 1]
    
    best_threshold = 0
    best_significance = 0
    
    for threshold in np.linspace(0.05, 0.95, num_thresholds):  
        y_pred = (y_scores >= threshold).astype(int)  

        signal = 0  # True Positives (TP)
        background = 0  # False Positives (FP)

        for event_type in event_weights.keys():
            weight = event_weights[event_type]  
            event_mask = event_test.values == event_type  

            # count weighted S (true positives) and B (false positives)
            signal += weight * sum((y_pred[event_mask] == 1) & (y_test.values[event_mask] == 1))  # TP
            background += weight * sum((y_pred[event_mask] == 1) & (y_test.values[event_mask] == 0))  # FP

        if signal + background > 0:
            signal_significance = signal / np.sqrt(signal + background)

            # track best threshold
            if signal_significance > best_significance:
                best_significance = signal_significance
                best_threshold = threshold
    
    # Compute final predictions using the best threshold
    y_pred_final = (y_scores >= best_threshold).astype(int)
    
    results = {
        "best_threshold": best_threshold,
        "best_significance": best_significance,
        "accuracy": accuracy_score(y_test, y_pred_final),
        "recall": recall_score(y_test, y_pred_final),
        "f1_score": f1_score(y_test, y_pred_final)
    }
    
    return results

In [ ]:
optimize_threshold(random_search_xgb.best_estimator_, X_test, y_test, event_test, event_weights, num_thresholds=50)

In [ ]:
def upload_to_gcs(bucket_name, source_file, destination_blob):
    """Uploads a file to Google Cloud Storage."""
    client = storage.Client()
    bucket = client.bucket(bucket_name)
    blob = bucket.blob(destination_blob)
    blob.upload_from_filename(source_file)
    print(f"✅ Model uploaded to GCS at: gs://{bucket_name}/{destination_blob}")

upload_to_gcs(BUCKET_NAME, "random_search_xgb.pkl", "models/random_search_xgb.pkl" )

In [ ]:
# lightgbm model

In [ ]:
import optuna
import lightgbm as lgb

def objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 50, 300),
        "max_depth": trial.suggest_int("max_depth", -1, 15),  
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.1, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 31, 100), 
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.7, 1.0),
        "subsample": trial.suggest_float("subsample", 0.7, 1.0),
        "reg_lambda": trial.suggest_float("reg_lambda", 1, 10, log=True),  
        "scale_pos_weight": scale_pos_weight, 
        "boosting_type": trial.suggest_categorical("boosting_type", ["gbdt", "dart"]),  
        "device": "gpu"  
    }

    X_train_sub, X_val, y_train_sub, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=50, stratify=y_train)

    model = lgb.LGBMClassifier(**params)
    model.fit(
        X_train_sub, y_train_sub,
        eval_set=[(X_val, y_val)],
        eval_metric="logloss",
        callbacks=[lgb.early_stopping(20, verbose=False)]  
    )

    y_pred = model.predict(X_val)
    recall = np.sum((y_pred == 1) & (y_val == 1)) / np.sum(y_val == 1)  # optimize for recall
    
    return recall

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50)  

print("Best LightGBM Parameters:", study.best_params)


In [ ]:
best_lgbm_params = { # copy and pasted from prev optimization study
    "n_estimators": 174,
    "max_depth": 0, 
    "learning_rate": 0.0822916699309106,
    "num_leaves": 69,
    "colsample_bytree": 0.8942260280757528,
    "subsample": 0.7992004746355641,
    "reg_lambda": 7.765769244511278,
    "boosting_type": "gbdt",
    "random_state": 42,
    "device": "gpu"  
}

lgbm_model = lgb.LGBMClassifier(**best_lgbm_params)
lgbm_model.fit(X_train, y_train)

with open("lgbm_model.pkl", "wb") as model_file:
    pickle.dump(lgbm_model, model_file)

upload_to_gcs(BUCKET_NAME, "lgbm_model.pkl", "models/lgbm_model.pkl")

In [ ]:
optimize_threshold(lgbm_model, X_test, y_test, event_test, event_weights, num_thresholds=50)

In [ ]:
# lightgbm (random forest) -- randomforestclassifier took too long to train

In [ ]:
def objective(trial):
    params = {
        "boosting_type": "rf",  # random forest
        "n_estimators": trial.suggest_int("n_estimators", 100, 500),
        "max_depth": trial.suggest_categorical("max_depth", [-1, 5, 10, 15]),  
        "num_leaves": trial.suggest_int("num_leaves", 20, 100),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.5, 1.0),  
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.5, 1.0),  
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 10), 
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.1),
        "reg_lambda": trial.suggest_loguniform("reg_lambda", 0.1, 10), 
        "scale_pos_weight": scale_pos_weight,  
        "device": "gpu",  
    }

    model = lgb.LGBMClassifier(**params)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    recall = np.sum((y_pred == 1) & (y_val == 1)) / np.sum(y_val == 1)  # optimize for recall
    return recall

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=20)  

print("Best LightGBM (RF) Parameters:", study.best_params_)

In [ ]:
lgbm_rf_params = {
    "n_estimators": 163,
    "max_depth": -1,
    "num_leaves": 100,
    "feature_fraction": 0.865132730509119, 
    "bagging_fraction": 0.522571778164637, 
    "bagging_freq": 8, 
    "learning_rate": 0.042053606399311175,
    "reg_lambda": 0.43456898133748256,  
    "boosting_type": "rf", 
    "random_state": 42,
    "n_jobs": -1
}

lgbm_rf_model = lgb.LGBMClassifier(**lgbm_rf_params)
lgbm_rf_model.fit(X_train, y_train)

with open("lgbm_rf_model.pkl", "wb") as model_file:
    pickle.dump(lgbm_rf_model, model_file)

upload_to_gcs(BUCKET_NAME, "lgbm_rf_model.pkl", "models/lgbm_rf_model.pkl")

In [ ]:
optimize_threshold(lgbm_rf_model, X_test, y_test, event_test, event_weights, num_thresholds=50)

In [ ]:
# mlp

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.utils.class_weight import compute_class_weight

# training in pytorch because it's faster
X_train_torch = torch.tensor(X_train.values, dtype=torch.float32).cuda()
y_train_torch = torch.tensor(y_train.values, dtype=torch.long).cuda()

class_weights = compute_class_weight(class_weight="balanced", classes=np.unique(y_train), y=y_train)
class_weights = torch.tensor(class_weights, dtype=torch.float32).cuda()

X_train_np = X_train_torch.cpu().numpy()  
y_train_np = y_train_torch.cpu().numpy()

X_train_sub, X_val, y_train_sub, y_val = train_test_split(
    X_train_np, y_train_np, test_size=0.1, random_state=42, stratify=y_train_np
)

X_train_sub = torch.tensor(X_train_sub, dtype=torch.float32).cuda()
y_train_sub = torch.tensor(y_train_sub, dtype=torch.long).cuda()
X_val = torch.tensor(X_val, dtype=torch.float32).cuda()
y_val = torch.tensor(y_val, dtype=torch.long).cuda()

class MLP(nn.Module):
    def __init__(self, input_dim, hidden1, hidden2, dropout):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden1)
        self.dropout1 = nn.Dropout(dropout)
        self.fc2 = nn.Linear(hidden1, hidden2)
        self.dropout2 = nn.Dropout(dropout)
        self.fc3 = nn.Linear(hidden2, 2)  

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.dropout1(x)
        x = torch.relu(self.fc2(x))
        x = self.dropout2(x)
        return self.fc3(x)  

def objective(trial):
    hidden1 = trial.suggest_int("hidden1", 128, 512)
    hidden2 = trial.suggest_int("hidden2", 64, 256)
    dropout = trial.suggest_float("dropout", 0.05, 0.3)  
    lr = trial.suggest_loguniform("learning_rate", 1e-4, 1e-3)
    l2_reg = trial.suggest_loguniform("l2_reg", 1e-5, 1e-3)

    model = MLP(X_train_sub.shape[1], hidden1, hidden2, dropout).cuda()
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=l2_reg)
    criterion = nn.CrossEntropyLoss(weight=class_weights).cuda()  

    for epoch in range(15):  # 15 epochs to reduce training time
        optimizer.zero_grad()
        outputs = model(X_train_sub)
        loss = criterion(outputs, y_train_sub)
        loss.backward()
        optimizer.step()

    model.eval()
    with torch.no_grad():
        outputs_val = model(X_val)
        probs = torch.softmax(outputs_val, dim=1)[:, 1]

        best_significance = 0
        best_threshold = 0
        for threshold in np.linspace(0.01, 0.99, 50):
            preds = (probs >= threshold).int()

            TP = (preds[y_val == 1] == 1).sum().item()  
            FP = (preds[y_val == 0] == 1).sum().item()  

            if TP + FP > 0:
                significance = TP / np.sqrt(TP + FP) 
                if significance > best_significance:
                    best_significance = significance
                    best_threshold = threshold
    
    return best_significance  # maximize signal significance

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=20)  # Run 20 trials

best_params = study.best_params
print("Best MLP Parameters:", best_params)

In [ ]:
from sklearn.neural_network import MLPClassifier

best_params_sklearn = {
    "hidden1": 317,  
    "hidden2": 122, 
    "alpha": 3.0687963534524235e-05, 
    "learning_rate_init": 0.0008421984089042774, 
}

mlp_sklearn = MLPClassifier(
    hidden_layer_sizes=(best_params_sklearn["hidden1"], best_params_sklearn["hidden2"]),
    activation="relu",
    solver="adam",
    alpha=best_params_sklearn["alpha"],  
    learning_rate_init=best_params_sklearn["learning_rate_init"],
    max_iter=100,  
    random_state=42,
    warm_start=True,
    early_stopping=True,  
    validation_fraction=0.1  
)

mlp_sklearn.fit(X_train, y_train)

with open("mlp_sklearn.pkl", "wb") as model_file:
    pickle.dump(mlp_sklearn, model_file)

upload_to_gcs(BUCKET_NAME, "mlp_sklearn.pkl", "models/mlp_sklearn.pkl")

In [ ]:
optimize_threshold(mlp_sklearn, X_test, y_test, event_test, event_weights, num_thresholds=50)

In [ ]:
# stacking model

In [ ]:
def load_model_from_gcs(bucket_name, model_path):
    """Loads a model from Google Cloud Storage."""
    client = storage.Client()
    bucket = client.bucket(bucket_name)
    blob = bucket.blob(model_path)

    model_bytes = blob.download_as_bytes()
    model = pickle.loads(model_bytes)
    print(f"Loaded model from gs://{bucket_name}/{model_path}")
    return model

lgbm_rf_model = load_model_from_gcs(BUCKET_NAME, "models/lgbm_rf_model.pkl")
lgbm_model = load_model_from_gcs(BUCKET_NAME, "models/lgbm_model.pkl")
xgb_model = load_model_from_gcs(BUCKET_NAME, "models/random_search_xgb.pkl")
mlp_model = load_model_from_gcs(BUCKET_NAME, "models/mlp_sklearn.pkl")

base_learners = {
    "LGBM_RF": lgbm_rf_model,
    "LGBM": lgbm_model,
    "XGBoost": xgb_model,
    "MLP": mlp_model
}

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import StackingClassifier

meta_xgb = XGBClassifier(
    n_estimators=100,
    max_depth=5,
    learning_rate=0.1,
    colsample_bytree=0.8,
    subsample=0.8,
    use_label_encoder=False,
    eval_metric="logloss",
    random_state=42
)

stacking_model = StackingClassifier(
    estimators=[(name, model) for name, model in base_learners.items()],
    final_estimator=meta_xgb,
    cv="prefit", # already trained models
    n_jobs=-1,  
    passthrough=False  # use only base model predictions as inputs to meta model
)

stacking_model.fit(X_train, y_train)

In [ ]:
optimize_threshold(stacking_model, X_test, y_test, event_test, event_weights, num_thresholds=50)

In [ ]:
with open("stacking_model.pkl", "wb") as model_file:
    pickle.dump(stacking_model, model_file)
    
upload_to_gcs(BUCKET_NAME, "stacking_model.pkl", "models/stacking_model.pkl")

In [ ]:
# plots

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

y_scores = stacking_model.predict_proba(X_test)[:, 1]  

signal_scores = y_scores[y_test == 1]
background_scores = y_scores[y_test == 0]

plt.figure(figsize=(10, 6))

sns.histplot(signal_scores, bins=50, color="red", alpha=0.5, label="Signal", kde=True, stat="count")
sns.histplot(background_scores, bins=50, color="blue", alpha=0.5, label="Background", kde=True, stat="count")

optimal_threshold = 0.932 # ADJUST FOR DIFFERENT TRIALS
plt.axvline(optimal_threshold, color="black", linestyle="dotted", linewidth=2, label=f"Optimal Threshold = {optimal_threshold}")

plt.xlabel("Predicted Probability")
plt.ylabel("Count")
plt.title("Stacking Model Output Probability Distribution")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
from sklearn.metrics import roc_curve, auc

fpr, tpr, _ = roc_curve(y_test, y_scores)
roc_auc = auc(fpr, tpr) 

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='blue', lw=2, label=f"ROC Curve (AUC = {roc_auc:.4f})")  
plt.plot([0, 1], [0, 1], color='gray', linestyle='--', lw=2)  
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Receiver Operating Characteristic (ROC) Curve")
plt.legend(loc="lower right")
plt.grid()
plt.show()

In [ ]:
y_scores = stacking_model.predict_proba(X_test)[:, 1]  
y_pred = (y_scores >= optimal_threshold).astype(int)  

true_positives = np.sum((y_pred == 1) & (y_test.values == 1))  
false_positives = np.sum((y_pred == 1) & (y_test.values == 0))  
false_negatives = np.sum((y_pred == 0) & (y_test.values == 1))  
true_negatives = np.sum((y_pred == 0) & (y_test.values == 0))  

tn_by_type = {event: 0 for event in np.unique(event_test.values)}  

for event_type in np.unique(event_test.values):
    event_mask = (event_test.values == event_type)
    tn_by_type[event_type] = np.sum((y_pred[event_mask] == 0) & (y_test.values[event_mask] == 0))  

print(f"True Positives (TP): {true_positives}")
print(f"False Positives (FP): {false_positives}")
print(f"False Negatives (FN): {false_negatives}")
print(f"True Negatives (TN): {true_negatives}")

print("\n True Negatives Per Background Type:")
for event_type, tn_count in tn_by_type.items():
    print(f"  {event_type}: {tn_count}")

conf_matrix = np.array([[true_negatives, false_positives],
                        [false_negatives, true_positives]])

conf_matrix_df = pd.DataFrame(conf_matrix, 
                              index=["Actual Negative", "Actual Positive"], 
                              columns=["Predicted Negative", "Predicted Positive"])

plt.figure(figsize=(6, 5))
sns.heatmap(conf_matrix_df, annot=True, fmt="d", cmap="Blues")
plt.title(f"Confusion Matrix (Threshold {optimal_threshold})")
plt.ylabel("Actual Label")
plt.xlabel("Predicted Label")
plt.show()